In [109]:
final_df = pd.read_excel('movie_link.xlsx') # 네이버 API로 받은 영화리뷰 링크

In [110]:
# 각 영화리뷰페이지 링크에서 영화코드 추출
import re
movie_link = final_df.loc[:,('title','link')]
movie_link['code'] = [re.sub(r'[^0-9]','',x) for x in movie_link['link']]

In [111]:
movie_link.head()

,title,link,code
0,베테랑,https://movie.naver.com/movie/bi/mi/basic.nhn?...,115977
1,암살,https://movie.naver.com/movie/bi/mi/basic.nhn?...,121048
2,어벤져스: 에이지 오브 울트론,https://movie.naver.com/movie/bi/mi/basic.nhn?...,98438
3,히말라야,https://movie.naver.com/movie/bi/mi/basic.nhn?...,100647
4,내부자들,https://movie.naver.com/movie/bi/mi/basic.nhn?...,121788


TEST: 베테랑 리뷰 20개 추출 (오래된 순)

In [124]:
from bs4 import BeautifulSoup
import requests
import time

title = '암살'
code = 121048
url = ("https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?"
       +"code="+str(code)
       +"&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N")

# 총 리뷰 개수 뽑기
response = requests.get(url)
html_text = response.text
soup = BeautifulSoup(html_text, 'html.parser')

num = int(soup.select("div.score_total > strong > em")[0].text.replace(',',''))

# 리뷰 개수 39,477인 경우 3943, 3944가 마지막 두 페이지 (테스트로 리뷰 20개 뽑기)
# 3945 페이지에는 39477을 10으로 나눈 나머지인 7개의 리뷰가 있음
page_lst = list(range(num//10-1,num//10+1))

In [125]:
page_lst

[3962, 3963]

In [142]:
movie_review_df = pd.DataFrame(columns=("title", "score", "review"))

In [143]:
# 각 페이지 별로 리뷰, 별점 추출
review = []
score = []

for page_num in page_lst:
    code = 121048
    url = ("https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?"
           +"code="+str(code)
           +"&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N"
           +"&order=newest"
           +"&page="+str(page_num)
           )
    response = requests.get(url)
    html_text = response.text
    soup = BeautifulSoup(html_text, 'html.parser')

    
    # 리뷰 추출
    review_lst = soup.select("div.score_reple > p")
    for elem in review_lst:
        review.append(elem.text.replace('\n','').replace('\t',''))
        
    # 별점 추출 ()
    score_lst = soup.select("div.star_score > em")
    for elem2 in score_lst:
        score.append(int(elem2.text))
    
    time.sleep(1)

In [144]:
len(score)

20

In [145]:
# 데이터 프레임에 저장
movie_review_df['review'] = review
movie_review_df['score'] = score
movie_review_df['title'] = title

movie_review_df.to_excel(title+'_review.xlsx')

## 영화 리뷰/별점 추출 코드

In [146]:
from bs4 import BeautifulSoup
import requests
import time

def movie_reviews(title, code, page_num=2):
    movie_review_df = pd.DataFrame(columns=("title", "score", "review"))
    url = ("https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?"
       +"code="+str(code)
       +"&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N")

    # 총 리뷰 개수 뽑기
    response = requests.get(url)
    html_text = response.text
    soup = BeautifulSoup(html_text, 'html.parser')

    num = int(soup.select("div.score_total > strong > em")[0].text.replace(',',''))

    # 리뷰 개수 39,477인 경우 3943, 3944가 마지막 두 페이지 (테스트로 리뷰 20개 뽑기)
    # 3945 페이지에는 39477을 10으로 나눈 나머지인 7개의 리뷰가 있음
    page_lst = list(range(num//10-page_num+1,num//10+1))
    
    
    # 각 페이지 별로 리뷰, 별점 추출
    review = []
    score = []

    for page in page_lst:
        url = ("https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?"
               +"code="+str(code)
               +"&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N"
               +"&order=newest"
               +"&page="+str(page)
               )
        response = requests.get(url)
        html_text = response.text
        soup = BeautifulSoup(html_text, 'html.parser')

        # 리뷰 추출
        review_lst = soup.select("div.score_reple > p")
        for elem in review_lst:
            review.append(elem.text.replace('\n','').replace('\t',''))

        # 별점 추출 ()
        score_lst = soup.select("div.star_score > em")
        for elem2 in score_lst:
            score.append(int(elem2.text))

        time.sleep(1)
        
    movie_review_df['review'] = review
    movie_review_df['score'] = score
    movie_review_df['title'] = title

    movie_review_df.to_excel(title+'_review.xlsx')
    
# 영화 3개 테스트
for i in range(3): 
    title, link, code = movie_link.iloc[i]
    # 영화 제목과 code 입력하면 영화별 엑셀파일 만듦.
    movie_reviews(title, code)